In [1]:
# python-dotenv 패키지: 환경 변수를 .env 파일에서 로드하는 라이브러리
# - 보안이 필요한 API 키 등을 관리하는데 사용
# - pip install python-dotenv로 설치 가능
from dotenv import load_dotenv

# load_dotenv(): .env 파일의 환경 변수를 현재 실행 환경에 로드하는 함수
# - 반환값: 성공 시 True, 실패 시 False
load_dotenv(override=True)

True

In [20]:
from langchain.prompts import PromptTemplate

template = """
당신은 클라우드 사용 매뉴얼을 기반으로 질문에 답변하는 전문가입니다.

다음은 검색된 문서 내용입니다:
====================
{context}
====================

위 내용을 참고하여 다음 질문에 정확하고 간결하게 답변하세요:
질문: {question}

- 검색된 문서의 내용을 벗어나지 마세요.

"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

In [30]:
from langchain.prompts import PromptTemplate

template = """
당신은 클라우드 사용 매뉴얼을 기반으로 질문에 답변하는 전문가입니다.

다음은 검색된 문서 내용입니다:
====================
{context}
====================

질문: {question}

위 내용을 참고하여 다음 조건을 지켜 답변하세요:

1. 반드시 마크다운 형식으로 정리하세요.
2. 단계별 설명이 필요하면 **숫자 목록**으로 나누어 설명하세요.
3. UI 경로는 `"Menu > Submenu"` 형식으로 표현하세요.
4. 전체 답변은 500자 이내로 간결하게 작성하세요.
5. 검색된 문서의 내용을 벗어나지 마세요.

답변:

"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

In [ ]:
from langchain.prompts import PromptTemplate

template = """
당신은 클라우드 사용 매뉴얼을 기반으로 질문에 답변하는 전문가입니다.

다음은 검색된 문서 내용입니다:
====================
{context}
====================

위 내용을 참고하여 다음 질문에 정확하고 간결하게 답변하세요:
질문: {question}

- 검색된 문서의 내용을 벗어나지 마세요.

"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

In [3]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
import os


# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# ChromaDB 벡터스토어 연결
# persist_directory는 ChromaDB가 저장된 디렉토리 경로
persist_directory = "./chroma_db"

# 기존 컬렉션에서 벡터스토어 로드
vectorstore = Chroma(
    collection_name="manual_user_collection",
    embedding_function=embeddings,
    persist_directory=persist_directory
)

print(f"벡터스토어가 성공적으로 로드되었습니다.")
print(f"컬렉션 이름: {vectorstore._collection.name}")
print(f"컬렉션 내 문서 수: {vectorstore._collection.count()}")

/var/folders/3k/glrtc_n92r1b88148508h8rc0000gn/T/ipykernel_65416/4021498024.py:7: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
/var/folders/3k/glrtc_n92r1b88148508h8rc0000gn/T/ipykernel_65416/4021498024.py:14: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectorstore = Chroma(


벡터스토어가 성공적으로 로드되었습니다.
컬렉션 이름: manual_user_collection
컬렉션 내 문서 수: 97


In [31]:
from langchain_community.llms import OpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda



# LLM 초기화
llm = ChatOpenAI(
            model="gpt-4o-mini",  # 사용할 모델 이름을 지정 가능
            temperature=0,        # temperature는 0~1 사이의 값으로, 0에 가까울수록 일관된 답변을, 1에 가까울수록 다양하고 창의적인 답변을 생성합니다
            max_tokens=100,       # 생성할 최대 토큰 수
            )

# Retriever 생성
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)


retriever_docs_backup = []
# section 필드만 추출하는 함수
def extract_section(docs):
    global retriever_docs_backup
    retriever_docs_backup = docs
    return {"context": [doc.page_content for doc in docs]}


# # 검색 쿼리 수행
# docs = retriever.invoke("네임스페이스 목록을 확인하고 싶어?")

# result = extract_section(docs)
# print("추출된 섹션 결과:", result)
# print()

# # 검색 결과 출력
# for i, doc in enumerate(docs):
#     print(f"\n--- 검색 결과 {i+1} ---")
#     print(f"내용: {doc.page_content}")
#     print(f"메타데이터: {doc.metadata}")    
# print('--------------------------------')


# for i, doc in enumerate(retriever_docs_backup):
#     print(f"\n--- 검색 결과 {i+1} ---")
#     print(f"내용: {doc.page_content}")
#     print(f"메타데이터: {doc.metadata}")    





# LCEL 체인 구성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 체인 테스트
question = "네임스페이스 목록을 확인하고 싶어."
response = chain.invoke(question)


print("질문:", question)
print("답변:", response)





질문: 네임스페이스 목록을 확인하고 싶어.
답변: 네임스페이스 목록을 확인하는 방법은 다음과 같습니다:

1. **Namespace 메뉴 진입**
   - `좌측 메뉴 > Namespaces` 메뉴를 클릭합니다.
   - 네임스페이스 화면에 진입하면, 현재 할당된 네임스페이스가 없을 경우 빈 리스트로 표시됩니다.

2. **네임스페이스 생성 확인**
   - 네임스페이스가 생성된 후, `신청관리 화면`에서


In [28]:
# Gradio는 머신러닝 모델을 위한 웹 인터페이스를 쉽게 만들 수 있게 해주는 파이썬 라이브러리입니다.
# 주요 특징:
# - 간단한 코드로 대화형 UI 생성 가능
# - 다양한 입출력 컴포넌트 제공 (텍스트, 이미지, 오디오 등)
# - 로컬 호스팅 및 Hugging Face Spaces 배포 지원
import gradio as gr

def answer_invoke(message, history):
    response = chain.invoke({"question": message})
    return response["answer"]


# Graiio 인터페이스 생성 
# Gradio의 ChatInterface 클래스를 사용하여 챗봇 인터페이스를 생성합니다
# - fn=answer_invoke: 사용자 입력을 처리할 콜백 함수를 지정합니다
# - title="QA Bot": 챗봇 UI의 제목을 설정합니다
demo = gr.ChatInterface(fn=answer_invoke, title="QA Bot")

# Graiio 실행  
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/Users/gu.han/Documents/AI.WORK/RAG_Master/langchain_env/lib/python3.11/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


In [27]:
demo.close()

Closing server running on port: 7860
